In [1]:
import tensorflow as tf

2023-11-03 16:30:00.336375: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-03 16:30:00.336409: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-03 16:30:00.336424: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-03 16:30:00.342102: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [41]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34, 31]

In [42]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [43]:
for sales in tf_dataset:
    print(sales)

tf.Tensor(21, shape=(), dtype=int32)
tf.Tensor(22, shape=(), dtype=int32)
tf.Tensor(-108, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)
tf.Tensor(-1, shape=(), dtype=int32)
tf.Tensor(32, shape=(), dtype=int32)
tf.Tensor(34, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)


In [44]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


In [45]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


In [69]:
  for sales in tf_dataset.take(3):
      print(sales.numpy())

1512
1584
2304


### Filtering data points

In [47]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


In [48]:
tf_dataset = tf_dataset.map(lambda x: x*72)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


In [49]:
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1584
1512
2232
2448
2232
2304


### Useful if running on multi gpu:

In [50]:
for sales in tf_dataset.batch(4).as_numpy_iterator():
    print(sales)

[1512 2232 1584 2304]
[2232 2448]


In [53]:
tf_dataset1 = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset1 = tf_dataset1.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(4)

In [54]:
for sales in tf_dataset1.as_numpy_iterator():
    print(sales)

[1512 2232 2304 1584]
[2232 2448]


# IMAGES

In [111]:
images_ds = tf.data.Dataset.list_files('images/*/*', shuffle=False)
for file in images_ds.take(3).as_numpy_iterator():
    print(file)

b'images/cat/360_F_409262227_YxFl54ykmDRYwPgd2YNLhfwPAF34Iqah.jpg'
b'images/cat/Cat_March_2010-1.jpg'
b'images/cat/Cat_November_2010-1a.jpg'


In [112]:
images_ds = tf.data.Dataset.list_files('images/*/*', shuffle=False)
for file in images_ds.take(3):
    print(file.numpy())

b'images/cat/360_F_409262227_YxFl54ykmDRYwPgd2YNLhfwPAF34Iqah.jpg'
b'images/cat/Cat_March_2010-1.jpg'
b'images/cat/Cat_November_2010-1a.jpg'


In [113]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3).as_numpy_iterator():
    print(file)

b'images/dog/Nala_by_Outdoor_Action_Photography_small.jpg'
b'images/cat/cat-scratching-furniture.jpg'
b'images/cat/June_odd-eyed-cat_cropped.jpg'


In [114]:
class_names = ['cat', 'dog']

In [115]:
image_count = len(images_ds)
image_count

26

In [116]:
train_size = int(image_count*0.8)

train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [117]:
len(train_ds)

20

In [118]:
len(test_ds)

6

## LABEL

In [119]:
def get_label(file_path):
    import os
    return tf.strings.split(file_path, os.path.sep)[-2]
    

In [120]:
def process_image(file_path):
    label = get_label(file_path)
    
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128,128])
    
    return img, label

In [121]:
for p in train_ds.take(3).as_numpy_iterator():
    print(p)

b'images/cat/Cat_November_2010-1a.jpg'
b'images/cat/June_odd-eyed-cat_cropped.jpg'
b'images/dog/Racer.jpg'


In [122]:
train_ds = train_ds.map(process_image)
for img, label in train_ds.take(3):
    print(f'Image: {img}')
    print(f'Label: {label}')

Image: [[[238.83203  237.83203  217.83203 ]
  [237.83203  236.83203  216.83203 ]
  [236.       235.       215.      ]
  ...
  [226.83203  227.83203  195.83203 ]
  [227.66406  226.66406  196.4961  ]
  [226.       226.       192.      ]]

 [[237.4961   236.4961   216.4961  ]
  [239.4961   238.4961   218.4961  ]
  [236.       235.       215.      ]
  ...
  [223.4961   224.4961   192.4961  ]
  [226.5039   225.5039   194.5039  ]
  [223.4961   223.4961   189.4961  ]]

 [[237.83984  236.83984  218.51953 ]
  [234.32031  233.32031  215.      ]
  [236.67969  235.67969  217.35938 ]
  ...
  [223.67969  224.67969  192.67969 ]
  [226.16016  225.16016  195.      ]
  [221.32031  221.32031  187.32031 ]]

 ...

 [[ 57.51953   39.51953   27.519531]
  [ 55.320312  37.320312  25.320312]
  [ 51.839844  33.839844  19.839844]
  ...
  [ 67.59766   46.597656  29.597656]
  [ 76.55859   50.558594  35.558594]
  [ 79.51953   53.51953   36.51953 ]]

 [[ 62.        44.        32.992188]
  [ 60.023438  41.03125   29.5

In [123]:
def normalize_img(img, label):
    return img/255, label

In [124]:
train_ds = train_ds.map(normalize_img)
for img, label in train_ds.take(5):
    print(f'Image: {img}')
    print(f'Label: {label}')

Image: [[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 ...

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]]
Label: b'cat'
Image: [[[0.9607843  0.9490196  0.9137255 ]
  [0.9607843  0.9490196  0.9137255 ]
  [0.9607843  0.9490196  0.9137255 ]
  ...
  [0.9764706  0.9647059  0.9372549 ]
  [0.97841215 0.96555984 0.9359336 ]
  [0.98039216 0.9647059  0.92941177]]

 [[0.9607843  0.9490196  0.9137255 ]
  [0.9607843  0.9490196  0.9137255 ]
  [0.9607843  0.9490196  0.9137255 ]
  ...
  [0.9764706  0.9647059  0.9372549 ]
  [0.9775582  0.9647059  0.93507963]
  [0.97786456 0.9621783  0.92